<a href="https://colab.research.google.com/github/Maoeyu/Github-Colab/blob/main/%E5%9F%BA%E4%BA%8EResNet50%E7%9A%84Fruit_Classifier_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

题目：基于 ResNet50 的水果分类

背景：使用基于卷积的深度神经网络 ResNet50 对 30 种水果进行分类

任务

划分训练集和验证集
按照 MMPreTrain CustomDataset 格式组织训练集和验证集
使用 MMPreTrain 算法库，编写配置文件，正确加载预训练模型
在水果数据集上进行微调训练
使用 MMPreTrain 的 ImageClassificationInferencer 接口，对网络水果图像，或自己拍摄的水果图像，使用训练好的模型进行分类
需提交的验证集评估指标（不能低于 60%）

ResNet-50


作业数据集下载： 链接：https://pan.baidu.com/s/1YgoU1M_v7ridtXB9xxbA1Q 提取码：52m9

课程中猫狗数据集下载地址： https://download.openmmlab.com/mmclassification/dataset/cats_dogs_dataset.tar

In [2]:
# 删掉原有的 mmpretrain 文件夹（如有）
!rm -rf mmpretrain

**1.MMPreTrain**

In [3]:
#事前工作 相关配置
!git clone https://github.com/open-mmlab/mmpretrain.git
%cd mmpretrain
!pip install -v -e .

Cloning into 'mmpretrain'...
remote: Enumerating objects: 16733, done.
remote: Counting objects: 100% (1564/1564), done.
remote: Compressing objects: 100% (675/675), done.
remote: Total 16733 (delta 986), reused 1313 (delta 870), pack-reused 15169
Receiving objects: 100% (16733/16733), 13.27 MiB | 25.12 MiB/s, done.
Resolving deltas: 100% (11607/11607), done.
/content/mmpretrain
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmpretrain
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-7sg5azac/mmpretrain.egg-info
  writing /tmp/pip-pip-egg-info-7sg5azac/mmpretrain.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-7sg5azac/mmpretrain.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-7sg5azac/mmpretrain.egg-info/requires.txt
  writing top-le

In [4]:
!pip install -U openmim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.3 MB/s eta 0:00:00


In [5]:
!mim install -e ".[multimodal]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/mmpretrain
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: mmpretrain
    Found existing installation: mmpretrain 1.0.0rc8
    Uninstalling mmpretrain-1.0.0rc8:
      Successfully uninstalled mmpretrain-1.0.0rc8
  Running setup.py develop for mmpretrain


In [6]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
%cd /content/mmpretrain

/content/mmpretrain


In [8]:
#创建data文件夹
!mkdir data

In [9]:
#从Google云端硬盘下载数据集fruit30_trian.tar
!gdown https://drive.google.com/u/0/uc?id=16AvoLcKzCfvsKFRVQ-RGVnsTRWzAe7ze&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=16AvoLcKzCfvsKFRVQ-RGVnsTRWzAe7ze
To: /content/mmpretrain/fruit30_train.tar
100% 187M/187M [00:07<00:00, 25.7MB/s]


In [10]:
#将下载的tar数据集 解压到data所对应的目录下
import tarfile

tar_path = "/content/mmpretrain/fruit30_train.tar"  # Path to your tar file
destination_path = "/content/mmpretrain/data/fruits"  # Path to the destination folder

with tarfile.open(tar_path) as tar:
    tar.extractall(path=destination_path)


**数据集划分**

In [11]:
import os
import shutil
import random
# File Location
data_path = "/content/mmpretrain/data/fruits"  # 源数据路径
train_path = "./data/train"  # 训练集路径
valid_path = "./data/valid"  # 验证集路径
test_path = "./data/test"  # 测试集路径
 
# 创建train, valid, test文件夹
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
 
# 遍历data文件夹下的每个子文件夹
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        # 创建相应的子文件夹
        os.makedirs(os.path.join(train_path, folder), exist_ok=True)
        os.makedirs(os.path.join(valid_path, folder), exist_ok=True)
        os.makedirs(os.path.join(test_path, folder), exist_ok=True)
 
        # 获取子文件夹中的文件列表
        files = os.listdir(folder_path)
        # 打乱文件顺序！！！！！！！！！！！！！！！
        random.shuffle(files)
 
        # 计算训练、验证、测试集的文件数量
        train_count = int(len(files) * 0.7)
        valid_count = int(len(files) * 0.2)
        test_count = len(files) - train_count - valid_count
 
        # 分别划分训练、验证、测试集
        train_files = files[:train_count]
        valid_files = files[train_count:(train_count+valid_count)]
        test_files = files[(train_count+valid_count):]
 
        # 移动文件
        for file in train_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(train_path, folder, file))
        for file in valid_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(valid_path, folder, file))
        for file in test_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(test_path, folder, file))
 
 

**Find Configs 文件**

In [12]:
!python /content/mmpretrain/tools/train.py /content/mmpretrain/configs/convmixer/convmixer-768-32_10xb64_in1k.py

06/07 16:05:46 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 1636626534
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50

In [27]:
!python /content/mmpretrain/tools/train.py /content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/convmixer-768-32_10xb64_in1k.py

06/07 17:19:59 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 196464682
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,

In [17]:
#推理
!python /content/mmpretrain/tools/test.py /content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/convmixer-768-32_10xb64_in1k.py /content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/epoch_5.pth
     

06/07 16:32:04 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 1207970244
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50

In [28]:
#测试效果
!python /content/mmpretrain/demo/image_demo.py /content/mmpretrain/data/test/山竹/103.jpg \
/content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/convmixer-768-32_10xb64_in1k.py \
--checkpoint /content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/epoch_2.pth --show --show-dir ./
     

Loads checkpoint by local backend from path: /content/mmpretrain/work_dirs/convmixer-768-32_10xb64_in1k/epoch_2.pth
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
06/07 17:51:33 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 23665 (\N{CJK UNIFIED IDEOGRAPH-5C71}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 31481 (\N{CJK UNIFIED IDEOGRAPH-7AF9}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 23665 (\N{CJK UNIFIED IDEOGRAPH-5C71}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarni